### Libraries

In [2]:
import glob
import os
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics

### Dataset

In [22]:
steamData = pd.read_csv('steam_data/steam.csv')
steamSupportInfo = pd.read_csv('steam_data/steam_support_info.csv')
steamChart = pd.read_csv('steam_data/steamCharts.csv')
steamSpyTagData = pd.read_csv('steam_data/steamspy_tag_data.csv')
steamGames = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_games/steam_games*.csv"))))
steamMediaData = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_media_data/steam_media_data*.csv"))))
steamRequirements = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_requirements/steam_requirements*.csv"))))
steamDescription = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_description/steam_description*.csv"))))

#game reviews
steamDayZReview = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_reviews/DayZ/steam_review_*.csv"))))
steamDeadbyDaylightReview = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_reviews/DeadbyDaylight/steam_review_*.csv"))))
steamFallout4Review = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_reviews/Fallout4/steam_review_*.csv"))))
steamNoMansSkyReview = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_reviews/NoMansSky/steam_review_*.csv"))))
steamPAYDAY2Review = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_reviews/PAYDAY2/steam_review_*.csv"))))
steamPLAYERUNKNOWNSBATTLEGROUNDSReview = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_reviews/PLAYERUNKNOWNSBATTLEGROUNDS/steam_review_*.csv"))))
steamSubnauticaReview = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_reviews/Subnautica/steam_review_*.csv"))))
steamTerrariaReview = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_reviews/Terraria/steam_review_*.csv"))))
steamTheElderScrollsVSkyrimReview = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_reviews/TheElderScrollsVSkyrim/steam_review_*.csv"))))
steam_HalfOfTheGameReviews = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_reviews/steam_review_*.csv"))))


C:\Users\Pyae Pyae\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\Pyae Pyae\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [25]:
for filename in glob.iglob("steam_data/steam_reviews" + '**/**', recursive=True):
     print(filename)

steam_data\steam_reviews\
steam_data\steam_reviews\DayZ
steam_data\steam_reviews\DayZ\steam_review_DayZ1.csv
steam_data\steam_reviews\DayZ\steam_review_DayZ2.csv
steam_data\steam_reviews\DeadbyDaylight
steam_data\steam_reviews\DeadbyDaylight\steam_review_DeadbyDaylight1.csv
steam_data\steam_reviews\DeadbyDaylight\steam_review_DeadbyDaylight2.csv
steam_data\steam_reviews\Fallout4
steam_data\steam_reviews\Fallout4\steam_review_Fallout41.csv
steam_data\steam_reviews\Fallout4\steam_review_Fallout42.csv
steam_data\steam_reviews\NoMansSky
steam_data\steam_reviews\NoMansSky\steam_review_NoMansSky1.csv
steam_data\steam_reviews\NoMansSky\steam_review_NoMansSky2.csv
steam_data\steam_reviews\NoMansSky\steam_review_NoMansSky3.csv
steam_data\steam_reviews\PAYDAY2
steam_data\steam_reviews\PAYDAY2\steam_review_PAYDAY21.csv
steam_data\steam_reviews\PAYDAY2\steam_review_PAYDAY22.csv
steam_data\steam_reviews\PLAYERUNKNOWNSBATTLEGROUNDS
steam_data\steam_reviews\PLAYERUNKNOWNSBATTLEGROUNDS\steam_review_PL